<a href="https://colab.research.google.com/github/xkzy/asterix-analytical-plot/blob/main/pcap_radar_analytic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

install dependency. run this only once.

# Install some dependency in first run.

In [ ]:
!pip install asterix-decoder
!pip install dpkt
!pip install ipyfilechooser
!pip install tqdm
!pip install pymap3d

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 253 kB 4.3 MB/s 
  Created wheel for asterix-decoder: filename=asterix_decoder-0.7.4-cp37-cp37m-linux_x86_64.whl size=935243 sha256=dc80a35e151ce333b40950e5e71f4fc571547e027484abea52cebedef59a1267
  Stored in directory: /root/.cache/pip/wheels/cc/b7/21/44066e6b355de1a6f2c21c33313a01867ff122d06e429b5002
Successfully built asterix-decoder
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 181 kB 4.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████

# Import dependency

In [ ]:
from google.colab import files
# from scapy.all import *
import asterix
import pandas as pd
import json
import dpkt
import os
import time
# from tqdm import tqdm
from tqdm.autonotebook import tqdm
import re
import collections
from IPython.display import HTML, display
from IPython.html import widgets
from ipyfilechooser import FileChooser
import pymap3d as pm
import plotly.express as px
datafile=None
uploaded=None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


# Load file to analysis.
upload files to analysis

In [ ]:
Files_source = "upload" #@param ["upload", "google_drive","skip"]
if Files_source=="upload":
    uploaded = files.upload()
    for fn in uploaded.keys():
      print('User uploaded file "{name}" with length {length} bytes'.format(
          name=fn, length=len(uploaded[fn])))
elif Files_source=="google_drive":
    from google.colab import drive
    drive.mount('/content/drive')


Saving ssr_bds_00230_20220726040000.pcap to ssr_bds_00230_20220726040000.pcap
User uploaded file "ssr_bds_00230_20220726040000.pcap" with length 3482956 bytes


or select file that have already uploaded.

In [63]:
#@title Select file to plot
# Create and display a FileChooser widget
fc = FileChooser(os.path.abspath(os.getcwd()))
# Set a file filter patern
fc.filter_pattern = '*.pcap'
display(fc)
uploaded=None

FileChooser(path='/content', filename='', title='', show_hidden=False, select_desc='Select', change_desc='Chan…

In [65]:
if uploaded != None:
    datafile = list(uploaded.keys())[0]
elif fc.selected != None:
    datafile = fc.selected
if datafile==None:
    raise Exception("Please select file.")

In [ ]:
df = pd.DataFrame()
# Read example file from packet resources
# sample_filename = asterix.get_sample_file(datafile)
f= open(datafile,'rb')
pcap = dpkt.pcap.Reader(f)
cntr=1
decryped_data=[]

for ts, buf in tqdm(pcap):
    eth = dpkt.ethernet.Ethernet(buf)
    if hasattr(eth, 'ip'):
        if hasattr(eth.ip, 'udp'):
            data = eth.ip.udp.data
            # hexdata = ":".join("{:02x}".format(ord(c)) for c in str(data))
            # print('Parsing packet %d : %s' %(cntr, hexdata))
            # print('Parsing packet %d' %(cntr))
            cntr += 1
            # Parse data
            parsed = asterix.parse(data)
            for item in parsed:
                if item['category']==48:
                    decryped_data.append(item)
        else:
            # print("tcp package")
            continue
    else:
        # print("no-ip package")
        continue


0it [00:00, ?it/s]

In [ ]:
decryped_data[0]

{'category': 48,
 'len': 73,
 'crc': '04AF08EB',
 'ts': 1660671185101.0,
 'hexdata': 'FDFF030484111C025AA01CC6556B0935027688122238F2F7C3282003ACF0000000000040FF550D1F00048350A8F98B19214C296006430C65F8CE047050184240F5084CB60D550D1702',
 'I010': {'SAC': {'desc': 'System Area Code', 'val': 132},
  'SIC': {'desc': 'System Identification Code', 'val': 17}},
 'I140': {'ToD': {'desc': 'Time Of Day', 'val': 14340.703125}},
 'I020': {'TYP': {'desc': 'TYP',
   'val': 5,
   'meaning': 'Single ModeS Roll-Call'},
  'SIM': {'desc': 'SIM', 'val': 0, 'meaning': 'Actual target report'},
  'RDP': {'desc': 'RDP', 'val': 0, 'meaning': 'Report from RDP Chain 1'},
  'SPI': {'desc': 'SPI', 'val': 0, 'meaning': 'Absence of SPI'},
  'RAB': {'desc': 'RAB',
   'val': 0,
   'meaning': 'Report from aircraft transponder'},
  'FX': {'desc': 'FX', 'val': 0, 'meaning': 'End of Data Item'}},
 'I040': {'RHO': {'desc': '', 'val': 28.7734375, 'max': 256.0},
  'THETA': {'desc': '', 'val': 120.1190185546875}},
 'I070': {'V'

flaten nested data

In [ ]:
def removekey(d, key):
    r = dict(d)
    del r[key]
    return r
def flatten(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            if k=="I250":
                lp=dict()
                
                cc=["40","50","60"]
                cl=[]
                for o in v:
                    if o["BDS"]["val"] in cc:
                        cl.append(o["BDS"]["val"])
                        lp["BDS_"+o["BDS"]["val"]]=removekey(o,"BDS")
                lp["BDS_count"]=len(cl)
                items.extend(flatten(lp, new_key, sep=sep).items())

            else:  
                for v2 in v:
                    new_key = parent_key + sep + k if parent_key else k
                    items.extend(flatten(v2, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [ ]:
flat_decryped_data=[flatten(i) for i in tqdm(decryped_data)]
df=pd.json_normalize(flat_decryped_data)

  0%|          | 0/9790 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if __name__ == '__main__':


In [ ]:
df.head()

,category,len,crc,ts,hexdata,I010.SAC.desc,I010.SAC.val,I010.SIC.desc,I010.SIC.val,I140.ToD.desc,I140.ToD.val,I020.TYP.desc,I020.TYP.val,I020.TYP.meaning,I020.SIM.desc,I020.SIM.val,I020.SIM.meaning,I020.RDP.desc,I020.RDP.val,I020.RDP.meaning,I020.SPI.desc,I020.SPI.val,I020.SPI.meaning,I020.RAB.desc,I020.RAB.val,I020.RAB.meaning,I020.FX.desc,I020.FX.val,I020.FX.meaning,I020.TST.desc,I020.TST.val,I020.TST.meaning,I020.spare.desc,I020.spare.val,I020.spare.const,I020.XPP.desc,I020.XPP.val,I020.XPP.meaning,I020.ME.desc,I020.ME.val,I020.ME.meaning,I020.MI.desc,I020.MI.val,I020.MI.meaning,I020.FOEFRI.desc,I020.FOEFRI.val,I020.FOEFRI.meaning,I040.RHO.desc,I040.RHO.val,I040.RHO.max,I040.THETA.desc,I040.THETA.val,I070.V.desc,I070.V.val,I070.V.meaning,I070.G.desc,I070.G.val,I070.G.meaning,I070.L.desc,I070.L.val,I070.L.meaning,I070.spare.desc,I070.spare.val,I070.spare.const,I070.Mode3A.desc,I070.Mode3A.val,I090.V.desc,I090.V.val,I090.V.meaning,I090.G.desc,I090.G.val,I090.G.meaning,I090.FL.desc,I090.FL.val,I220.ACAddr.desc,I220.ACAddr.val,I240.TId.desc,I240.TId.val,I230.COM.desc,I230.COM.val,I230.STAT.desc,I230.STAT.val,I230.SI.desc,I230.SI.val,I230.SI.meaning,I230.spare.desc,I230.spare.val,I230.spare.const,I230.ModeSSSC.desc,I230.ModeSSSC.val,I230.ModeSSSC.meaning,I230.ARC.desc,I230.ARC.val,I230.ARC.meaning,I230.AIC.desc,I230.AIC.val,I230.AIC.meaning,I230.BDS16.desc,I230.BDS16.val,I230.BDS37.desc,I230.BDS37.val,I250.BDS_40.MCP_ALT_STATUS.desc,I250.BDS_40.MCP_ALT_STATUS.val,I250.BDS_40.MCP_ALT.desc,I250.BDS_40.MCP_ALT.val,I250.BDS_40.FMS_ALT_STATUS.desc,I250.BDS_40.FMS_ALT_STATUS.val,I250.BDS_40.FMS_ALT.desc,I250.BDS_40.FMS_ALT.val,I250.BDS_40.BP_STATUS.desc,I250.BDS_40.BP_STATUS.val,I250.BDS_40.BP.desc,I250.BDS_40.BP.val,I250.BDS_40.res1.desc,I250.BDS_40.res1.val,I250.BDS_40.MODE_STATUS.desc,I250.BDS_40.MODE_STATUS.val,I250.BDS_40.VNAV.desc,I250.BDS_40.VNAV.val,I250.BDS_40.VNAV.meaning,I250.BDS_40.ALT_HOLD.desc,I250.BDS_40.ALT_HOLD.val,I250.BDS_40.ALT_HOLD.meaning,I250.BDS_40.APP.desc,I250.BDS_40.APP.val,I250.BDS_40.APP.meaning,I250.BDS_40.res2.desc,I250.BDS_40.res2.val,I250.BDS_40.TARGET_ALT_STATUS.desc,I250.BDS_40.TARGET_ALT_STATUS.val,I250.BDS_40.TARGET_ALT_STATUS.meaning,I250.BDS_40.TARGET_ALT_SOURCE.desc,I250.BDS_40.TARGET_ALT_SOURCE.val,I250.BDS_40.TARGET_ALT_SOURCE.meaning,I250.BDS_50.RA_STATUS.desc,I250.BDS_50.RA_STATUS.val,I250.BDS_50.RA.desc,I250.BDS_50.RA.val,I250.BDS_50.TTA_STATUS.desc,I250.BDS_50.TTA_STATUS.val,I250.BDS_50.TTA.desc,I250.BDS_50.TTA.val,I250.BDS_50.GS_STATUS.desc,I250.BDS_50.GS_STATUS.val,I250.BDS_50.GS.desc,I250.BDS_50.GS.val,I250.BDS_50.TAR_STATUS.desc,I250.BDS_50.TAR_STATUS.val,I250.BDS_50.TAR.desc,I250.BDS_50.TAR.val,I250.BDS_50.TAS_STATUS.desc,I250.BDS_50.TAS_STATUS.val,I250.BDS_50.TAS.desc,I250.BDS_50.TAS.val,I250.BDS_60.HDG_STATUS.desc,I250.BDS_60.HDG_STATUS.val,I250.BDS_60.HDG.desc,I250.BDS_60.HDG.val,I250.BDS_60.IAS_STAT.desc,I250.BDS_60.IAS_STAT.val,I250.BDS_60.IAS.desc,I250.BDS_60.IAS.val,I250.BDS_60.MACH_STATUS.desc,I250.BDS_60.MACH_STATUS.val,I250.BDS_60.MACH.desc,I250.BDS_60.MACH.val,I250.BDS_60.BAR_STATUS.desc,I250.BDS_60.BAR_STATUS.val,I250.BDS_60.BAR.desc,I250.BDS_60.BAR.val,I250.BDS_60.IVV_STATUS.desc,I250.BDS_60.IVV_STATUS.val,I250.BDS_60.IVV.desc,I250.BDS_60.IVV.val,I250.BDS_count,I161.Tn.desc,I161.Tn.val,I042.X.desc,I042.X.val,I042.Y.desc,I042.Y.val,I200.CGS.desc,I200.CGS.val,I200.CHdg.desc,I200.CHdg.val,I170.CNF.desc,I170.CNF.val,I170.CNF.meaning,I170.RAD.desc,I170.RAD.val,I170.RAD.meaning,I170.DOU.desc,I170.DOU.val,I170.DOU.meaning,I170.MAH.desc,I170.MAH.val,I170.MAH.meaning,I170.CDM.desc,I170.CDM.val,I170.CDM.meaning,I170.FX.desc,I170.FX.val,I170.FX.meaning,ISP.SP.desc,ISP.SP.val,I170.TRE.desc,I170.TRE.val,I170.TRE.meaning,I170.GHO.desc,I170.GHO.val,I170.GHO.meaning,I170.SUP.desc,I170.SUP.val,I170.SUP.meaning,I170.TCC.desc,I170.TCC.val,I170.TCC.meaning,I170.spare.desc,I170.spare.val,I170.spare.const
0,48,29,359266AF,1.660643e+12,FDC10284110103F0A3808D04FE490B6D04B08800004C5505C35C350000,

In [ ]:
a=df.columns.values
print(a)

['category' 'len' 'crc' 'ts' 'hexdata' 'I010.SAC.desc' 'I010.SAC.val'
 'I010.SIC.desc' 'I010.SIC.val' 'I140.ToD.desc' 'I140.ToD.val'
 'I020.TYP.desc' 'I020.TYP.val' 'I020.TYP.meaning' 'I020.SIM.desc'
 'I020.SIM.val' 'I020.SIM.meaning' 'I020.RDP.desc' 'I020.RDP.val'
 'I020.RDP.meaning' 'I020.SPI.desc' 'I020.SPI.val' 'I020.SPI.meaning'
 'I020.RAB.desc' 'I020.RAB.val' 'I020.RAB.meaning' 'I020.FX.desc'
 'I020.FX.val' 'I020.FX.meaning' 'I040.RHO.desc' 'I040.RHO.val'
 'I040.RHO.max' 'I040.THETA.desc' 'I040.THETA.val' 'I070.V.desc'
 'I070.V.val' 'I070.V.meaning' 'I070.G.desc' 'I070.G.val' 'I070.G.meaning'
 'I070.L.desc' 'I070.L.val' 'I070.L.meaning' 'I070.spare.desc'
 'I070.spare.val' 'I070.spare.const' 'I070.Mode3A.desc' 'I070.Mode3A.val'
 'I090.V.desc' 'I090.V.val' 'I090.V.meaning' 'I090.G.desc' 'I090.G.val'
 'I090.G.meaning' 'I090.FL.desc' 'I090.FL.val' 'I220.ACAddr.desc'
 'I220.ACAddr.val' 'I240.TId.desc' 'I240.TId.val'
 'I250.BDS_40.MCP_ALT_STATUS.desc' 'I250.BDS_40.MCP_ALT_STATUS.val'
 

In [ ]:
p = re.compile('.+(\.desc)')
desc_col = [ s for s in a if p.match(s) ]
print(desc_col)

['I010.SAC.desc', 'I010.SIC.desc', 'I140.ToD.desc', 'I020.TYP.desc', 'I020.SIM.desc', 'I020.RDP.desc', 'I020.SPI.desc', 'I020.RAB.desc', 'I020.FX.desc', 'I040.RHO.desc', 'I040.THETA.desc', 'I070.V.desc', 'I070.G.desc', 'I070.L.desc', 'I070.spare.desc', 'I070.Mode3A.desc', 'I090.V.desc', 'I090.G.desc', 'I090.FL.desc', 'I220.ACAddr.desc', 'I240.TId.desc', 'I250.BDS_40.MCP_ALT_STATUS.desc', 'I250.BDS_40.MCP_ALT.desc', 'I250.BDS_40.FMS_ALT_STATUS.desc', 'I250.BDS_40.FMS_ALT.desc', 'I250.BDS_40.BP_STATUS.desc', 'I250.BDS_40.BP.desc', 'I250.BDS_40.res1.desc', 'I250.BDS_40.MODE_STATUS.desc', 'I250.BDS_40.VNAV.desc', 'I250.BDS_40.ALT_HOLD.desc', 'I250.BDS_40.APP.desc', 'I250.BDS_40.res2.desc', 'I250.BDS_40.TARGET_ALT_STATUS.desc', 'I250.BDS_40.TARGET_ALT_SOURCE.desc', 'I250.BDS_50.RA_STATUS.desc', 'I250.BDS_50.RA.desc', 'I250.BDS_50.TTA_STATUS.desc', 'I250.BDS_50.TTA.desc', 'I250.BDS_50.GS_STATUS.desc', 'I250.BDS_50.GS.desc', 'I250.BDS_50.TAR_STATUS.desc', 'I250.BDS_50.TAR.desc', 'I250.BDS_50.

In [ ]:
df_reduced=df.copy()
items2drop=["len", "crc","hexdata"]
items2drop.extend(desc_col)
rename_mapper={"I010.SAC.val": "System Area Code", 
               "I010.SIC.val": "System Identification Code",
               "I000.MsgTyp.meaning":"Message Type",
               "I030.ToD.val":"Time of day",
               "I020.Azi.val":"Antenna azimuth"}
df_reduced.drop(items2drop, axis = 1, inplace=True)
# df_reduced=df_reduced.rename(columns=rename_mapper)
df_reduced['I140.ToD.val'] = df_reduced['I140.ToD.val'].astype('timedelta64[s]')
# df_reduced['I140.ToD.val'] =  pd.to_datetime(df_reduced['I140.ToD.val'], format='%H:%M:%S.%f')
# df_reduced['I140.ToD.val'] =  pd.to_timedelta(df_reduced['I140.ToD.val'])

df_reduced['ts'] = df_reduced['ts'].astype('int64')
df_reduced['I250.BDS_count'] = df_reduced['I250.BDS_count'].fillna(0)
df_reduced['I250.BDS_count'] = df_reduced['I250.BDS_count'].astype('int8')
df_reduced.dropna( axis=1, how="all")
## feet corection
df_reduced['I090.FL.val']=df_reduced['I090.FL.val']*100
## convert NM to meters
df_reduced["enu_e"]=df_reduced['I042.X.val'].apply(lambda x: x * 1852)
df_reduced["enu_n"]=df_reduced['I042.Y.val'].apply(lambda x: x * 1852)
df_reduced["enu_u"]=df_reduced['I090.FL.val'].apply(lambda x: x * 0.3048)

df_reduced.head()

,category,ts,I010.SAC.val,I010.SIC.val,I140.ToD.val,I020.TYP.val,I020.TYP.meaning,I020.SIM.val,I020.SIM.meaning,I020.RDP.val,I020.RDP.meaning,I020.SPI.val,I020.SPI.meaning,I020.RAB.val,I020.RAB.meaning,I020.FX.val,I020.FX.meaning,I040.RHO.val,I040.RHO.max,I040.THETA.val,I070.V.val,I070.V.meaning,I070.G.val,I070.G.meaning,I070.L.val,I070.L.meaning,I070.spare.val,I070.spare.const,I070.Mode3A.val,I090.V.val,I090.V.meaning,I090.G.val,I090.G.meaning,I090.FL.val,I220.ACAddr.val,I240.TId.val,I250.BDS_40.MCP_ALT_STATUS.val,I250.BDS_40.MCP_ALT.val,I250.BDS_40.FMS_ALT_STATUS.val,I250.BDS_40.FMS_ALT.val,I250.BDS_40.BP_STATUS.val,I250.BDS_40.BP.val,I250.BDS_40.res1.val,I250.BDS_40.MODE_STATUS.val,I250.BDS_40.VNAV.val,I250.BDS_40.VNAV.meaning,I250.BDS_40.ALT_HOLD.val,I250.BDS_40.ALT_HOLD.meaning,I250.BDS_40.APP.val,I250.BDS_40.APP.meaning,I250.BDS_40.res2.val,I250.BDS_40.TARGET_ALT_STATUS.val,I250.BDS_40.TARGET_ALT_STATUS.meaning,I250.BDS_40.TARGET_ALT_SOURCE.val,I250.BDS_40.TARGET_ALT_SOURCE.meaning,I250.BDS_50.RA_STATUS.val,I250.BDS_50.RA.val,I250.BDS_50.TTA_STATUS.val,I250.BDS_50.TTA.val,I250.BDS_50.GS_STATUS.val,I250.BDS_50.GS.val,I250.BDS_50.TAR_STATUS.val,I250.BDS_50.TAR.val,I250.BDS_50.TAS_STATUS.val,I250.BDS_50.TAS.val,I250.BDS_60.HDG_STATUS.val,I250.BDS_60.HDG.val,I250.BDS_60.IAS_STAT.val,I250.BDS_60.IAS.val,I250.BDS_60.MACH_STATUS.val,I250.BDS_60.MACH.val,I250.BDS_60.BAR_STATUS.val,I250.BDS_60.BAR.val,I250.BDS_60.IVV_STATUS.val,I250.BDS_60.IVV.val,I250.BDS_count,I161.Tn.val,I042.X.val,I042.Y.val,I200.CGS.val,I200.CHdg.val,I170.CNF.val,I170.CNF.meaning,I170.RAD.val,I170.RAD.meaning,I170.DOU.val,I170.DOU.meaning,I170.MAH.val,I170.MAH.meaning,I170.CDM.val,I170.CDM.meaning,I170.FX.val,I170.FX.meaning,I230.COM.val,I230.STAT.val,I230.SI.val,I230.SI.meaning,I230.spare.val,I230.spare.const,I230.ModeSSSC.val,I230.ModeSSSC.meaning,I230.ARC.val,I230.ARC.meaning,I230.AIC.val,I230.AIC.meaning,I230.BDS16.val,I230.BDS37.val,ISP.SP.val,I020.TST.val,I020.TST.meaning,I020.spare.val,I020.spare.const,I020.XPP.val,I020.XPP.meaning,I020.ME.val,I020.ME.meaning,I020.MI.val,I020.MI.meaning,I020.FOEFRI.val,I020.FOEFRI.meaning,I170.TRE.val,I170.TRE.meaning,I170.GHO.val,I170.GHO.meaning,I170.SUP.val,I170.SUP.meaning,I170.TCC.val,I170.TCC.meaning,I170.spare.val,I170.spare.const,enu_e,enu_n,enu_u,wgs84_lat,wgs84_lon,wgs84_alt
0,48,1660671185101,132,17,0 days 03:59:00,5,Single ModeS Roll-Call,0,Actual target report,0,Report from RDP Chain 1,0,Absence of SPI,0,Report from aircraft transponder,0,End of Data Item,28.773438,256.0,120.119019,0.0,Code validated,0.0,Default,0.0,Mode-3/A code as derived from the reply of the transponder,0.0,0.0,4465,0.0,Code validated,0.0,Default,15750.0,881222,NOK702,1.0,23008.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not active,0.0,Not active,0.0,Not active,0.0,0.0,No source information provided,0.0,Unknown,1.0,-1.054688,1.0,113.554688,1.0,248.0,0.0,0.00000,1.0,262.0,1.0,115.136719,1.0,197.0,1.0,0.400,1.0,1312.0,1.0,1312.0,3,1603.0,24.789062,-14.390625,249.609375,112.631836,0.0,Confirmed Track,2.0,SSR/ModeS Track,0.0,Normal confidence,0.0,No horizontal man. sensed,1.0,Climbing,0.0,End of Data Item,2.0,0.0,0.0,SI-Code Capable,0.0,0.0,1.0,Yes,1.0,25ft resolution,1.0,Yes,1.0,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45909.34375,-26651.43750,4800.60,18.518570,99.401494,5310.209492
1,48,1660671185102,132,17,0 days 03:59:00,5,Single ModeS Roll-Call,0,Actual target report,1,Report from RDP Chain 2,0,Absence of SPI,0,Report from aircraft transponder,0,End of Data Item,28.773438,256.0,120.119019,0.0,Code validated,0.0,Default,0.0,Mode-3/A code as derived from the reply of the transponder,0.0,0.0,4465,0.0,Code validated,0.0,Default,15750.0,881222,NOK702,1.0,23008.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not active,0.0,Not active,0.0,Not active,0.0,0.0,No source information provided,0.0,Unknown,1.0,-1.054688,1.0,113.554688,1.0,248.0,0.0,0.00000,1.0,262.0,1.0,115.136719,1.0,197.0,1.0,0.400,1.0,1312.0,1.0,1312.0,3,

In [ ]:
print(df_reduced.dtypes)

category                                 int64         
ts                                       int64         
I010.SAC.val                             int64         
I010.SIC.val                             int64         
I140.ToD.val                             datetime64[ns]
I020.TYP.val                             int64         
I020.TYP.meaning                         object        
I020.SIM.val                             int64         
I020.SIM.meaning                         object        
I020.RDP.val                             int64         
I020.RDP.meaning                         object        
I020.SPI.val                             int64         
I020.SPI.meaning                         object        
I020.RAB.val                             int64         
I020.RAB.meaning                         object        
I020.FX.val                              int64         
I020.FX.meaning                          object        
I040.RHO.val                             float64

In [ ]:
print(df_reduced.isnull().sum())

category                                 0   
ts                                       0   
I010.SAC.val                             0   
I010.SIC.val                             0   
I140.ToD.val                             0   
I020.TYP.val                             0   
I020.TYP.meaning                         0   
I020.SIM.val                             0   
I020.SIM.meaning                         0   
I020.RDP.val                             0   
I020.RDP.meaning                         0   
I020.SPI.val                             0   
I020.SPI.meaning                         0   
I020.RAB.val                             0   
I020.RAB.meaning                         0   
I020.FX.val                              0   
I020.FX.meaning                          0   
I040.RHO.val                             34  
I040.RHO.max                             34  
I040.THETA.val                           34  
I070.V.val                               194 
I070.V.meaning                    

In [ ]:
# tackconfirmed=df_reduced.loc[df_reduced['I161.Tn.val'] != Null]
tackconfirmed=df_reduced[df_reduced['I161.Tn.val'].notnull()]
tackconfirmed.head()

,category,ts,I010.SAC.val,I010.SIC.val,I140.ToD.val,I020.TYP.val,I020.TYP.meaning,I020.SIM.val,I020.SIM.meaning,I020.RDP.val,I020.RDP.meaning,I020.SPI.val,I020.SPI.meaning,I020.RAB.val,I020.RAB.meaning,I020.FX.val,I020.FX.meaning,I040.RHO.val,I040.RHO.max,I040.THETA.val,I070.V.val,I070.V.meaning,I070.G.val,I070.G.meaning,I070.L.val,I070.L.meaning,I070.spare.val,I070.spare.const,I070.Mode3A.val,I090.V.val,I090.V.meaning,I090.G.val,I090.G.meaning,I090.FL.val,I220.ACAddr.val,I240.TId.val,I250.BDS_40.MCP_ALT_STATUS.val,I250.BDS_40.MCP_ALT.val,I250.BDS_40.FMS_ALT_STATUS.val,I250.BDS_40.FMS_ALT.val,I250.BDS_40.BP_STATUS.val,I250.BDS_40.BP.val,I250.BDS_40.res1.val,I250.BDS_40.MODE_STATUS.val,I250.BDS_40.VNAV.val,I250.BDS_40.VNAV.meaning,I250.BDS_40.ALT_HOLD.val,I250.BDS_40.ALT_HOLD.meaning,I250.BDS_40.APP.val,I250.BDS_40.APP.meaning,I250.BDS_40.res2.val,I250.BDS_40.TARGET_ALT_STATUS.val,I250.BDS_40.TARGET_ALT_STATUS.meaning,I250.BDS_40.TARGET_ALT_SOURCE.val,I250.BDS_40.TARGET_ALT_SOURCE.meaning,I250.BDS_50.RA_STATUS.val,I250.BDS_50.RA.val,I250.BDS_50.TTA_STATUS.val,I250.BDS_50.TTA.val,I250.BDS_50.GS_STATUS.val,I250.BDS_50.GS.val,I250.BDS_50.TAR_STATUS.val,I250.BDS_50.TAR.val,I250.BDS_50.TAS_STATUS.val,I250.BDS_50.TAS.val,I250.BDS_60.HDG_STATUS.val,I250.BDS_60.HDG.val,I250.BDS_60.IAS_STAT.val,I250.BDS_60.IAS.val,I250.BDS_60.MACH_STATUS.val,I250.BDS_60.MACH.val,I250.BDS_60.BAR_STATUS.val,I250.BDS_60.BAR.val,I250.BDS_60.IVV_STATUS.val,I250.BDS_60.IVV.val,I250.BDS_count,I161.Tn.val,I042.X.val,I042.Y.val,I200.CGS.val,I200.CHdg.val,I170.CNF.val,I170.CNF.meaning,I170.RAD.val,I170.RAD.meaning,I170.DOU.val,I170.DOU.meaning,I170.MAH.val,I170.MAH.meaning,I170.CDM.val,I170.CDM.meaning,I170.FX.val,I170.FX.meaning,I230.COM.val,I230.STAT.val,I230.SI.val,I230.SI.meaning,I230.spare.val,I230.spare.const,I230.ModeSSSC.val,I230.ModeSSSC.meaning,I230.ARC.val,I230.ARC.meaning,I230.AIC.val,I230.AIC.meaning,I230.BDS16.val,I230.BDS37.val,ISP.SP.val,I020.TST.val,I020.TST.meaning,I020.spare.val,I020.spare.const,I020.XPP.val,I020.XPP.meaning,I020.ME.val,I020.ME.meaning,I020.MI.val,I020.MI.meaning,I020.FOEFRI.val,I020.FOEFRI.meaning,I170.TRE.val,I170.TRE.meaning,I170.GHO.val,I170.GHO.meaning,I170.SUP.val,I170.SUP.meaning,I170.TCC.val,I170.TCC.meaning,I170.spare.val,I170.spare.const,enu_e,enu_n,enu_u,wgs84_lat,wgs84_lon,wgs84_alt
0,48,1660671185101,132,17,1970-01-01 03:59:00,5,Single ModeS Roll-Call,0,Actual target report,0,Report from RDP Chain 1,0,Absence of SPI,0,Report from aircraft transponder,0,End of Data Item,28.773438,256.0,120.119019,0.0,Code validated,0.0,Default,0.0,Mode-3/A code as derived from the reply of the transponder,0.0,0.0,4465,0.0,Code validated,0.0,Default,15750.0,881222,NOK702,1.0,23008.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not active,0.0,Not active,0.0,Not active,0.0,0.0,No source information provided,0.0,Unknown,1.0,-1.054688,1.0,113.554688,1.0,248.0,0.0,0.00000,1.0,262.0,1.0,115.136719,1.0,197.0,1.0,0.400,1.0,1312.0,1.0,1312.0,3,1603.0,24.789062,-14.390625,249.609375,112.631836,0.0,Confirmed Track,2.0,SSR/ModeS Track,0.0,Normal confidence,0.0,No horizontal man. sensed,1.0,Climbing,0.0,End of Data Item,2.0,0.0,0.0,SI-Code Capable,0.0,0.0,1.0,Yes,1.0,25ft resolution,1.0,Yes,1.0,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45909.34375,-26651.43750,4800.60,18.518570,99.401494,5310.209492
1,48,1660671185102,132,17,1970-01-01 03:59:00,5,Single ModeS Roll-Call,0,Actual target report,1,Report from RDP Chain 2,0,Absence of SPI,0,Report from aircraft transponder,0,End of Data Item,28.773438,256.0,120.119019,0.0,Code validated,0.0,Default,0.0,Mode-3/A code as derived from the reply of the transponder,0.0,0.0,4465,0.0,Code validated,0.0,Default,15750.0,881222,NOK702,1.0,23008.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not active,0.0,Not active,0.0,Not active,0.0,0.0,No source information provided,0.0,Unknown,1.0,-1.054688,1.0,113.554688,1.0,248.0,0.0,0.00000,1.0,262.0,1.0,115.136719,1.0,197.0,1.0,0.400,1.0,1312.0,1.0,1

In [ ]:
print(tackconfirmed.isnull().sum())

category                                 0   
ts                                       0   
I010.SAC.val                             0   
I010.SIC.val                             0   
I140.ToD.val                             0   
I020.TYP.val                             0   
I020.TYP.meaning                         0   
I020.SIM.val                             0   
I020.SIM.meaning                         0   
I020.RDP.val                             0   
I020.RDP.meaning                         0   
I020.SPI.val                             0   
I020.SPI.meaning                         0   
I020.RAB.val                             0   
I020.RAB.meaning                         0   
I020.FX.val                              0   
I020.FX.meaning                          0   
I040.RHO.val                             34  
I040.RHO.max                             34  
I040.THETA.val                           34  
I070.V.val                               34  
I070.V.meaning                    

In [ ]:
tackconfirmed.shape

(8820, 136)

In [ ]:
tackconfirmed.head()

,category,ts,I010.SAC.val,I010.SIC.val,I140.ToD.val,I020.TYP.val,I020.TYP.meaning,I020.SIM.val,I020.SIM.meaning,I020.RDP.val,I020.RDP.meaning,I020.SPI.val,I020.SPI.meaning,I020.RAB.val,I020.RAB.meaning,I020.FX.val,I020.FX.meaning,I040.RHO.val,I040.RHO.max,I040.THETA.val,I070.V.val,I070.V.meaning,I070.G.val,I070.G.meaning,I070.L.val,I070.L.meaning,I070.spare.val,I070.spare.const,I070.Mode3A.val,I090.V.val,I090.V.meaning,I090.G.val,I090.G.meaning,I090.FL.val,I220.ACAddr.val,I240.TId.val,I250.BDS_40.MCP_ALT_STATUS.val,I250.BDS_40.MCP_ALT.val,I250.BDS_40.FMS_ALT_STATUS.val,I250.BDS_40.FMS_ALT.val,I250.BDS_40.BP_STATUS.val,I250.BDS_40.BP.val,I250.BDS_40.res1.val,I250.BDS_40.MODE_STATUS.val,I250.BDS_40.VNAV.val,I250.BDS_40.VNAV.meaning,I250.BDS_40.ALT_HOLD.val,I250.BDS_40.ALT_HOLD.meaning,I250.BDS_40.APP.val,I250.BDS_40.APP.meaning,I250.BDS_40.res2.val,I250.BDS_40.TARGET_ALT_STATUS.val,I250.BDS_40.TARGET_ALT_STATUS.meaning,I250.BDS_40.TARGET_ALT_SOURCE.val,I250.BDS_40.TARGET_ALT_SOURCE.meaning,I250.BDS_50.RA_STATUS.val,I250.BDS_50.RA.val,I250.BDS_50.TTA_STATUS.val,I250.BDS_50.TTA.val,I250.BDS_50.GS_STATUS.val,I250.BDS_50.GS.val,I250.BDS_50.TAR_STATUS.val,I250.BDS_50.TAR.val,I250.BDS_50.TAS_STATUS.val,I250.BDS_50.TAS.val,I250.BDS_60.HDG_STATUS.val,I250.BDS_60.HDG.val,I250.BDS_60.IAS_STAT.val,I250.BDS_60.IAS.val,I250.BDS_60.MACH_STATUS.val,I250.BDS_60.MACH.val,I250.BDS_60.BAR_STATUS.val,I250.BDS_60.BAR.val,I250.BDS_60.IVV_STATUS.val,I250.BDS_60.IVV.val,I250.BDS_count,I161.Tn.val,I042.X.val,I042.Y.val,I200.CGS.val,I200.CHdg.val,I170.CNF.val,I170.CNF.meaning,I170.RAD.val,I170.RAD.meaning,I170.DOU.val,I170.DOU.meaning,I170.MAH.val,I170.MAH.meaning,I170.CDM.val,I170.CDM.meaning,I170.FX.val,I170.FX.meaning,I230.COM.val,I230.STAT.val,I230.SI.val,I230.SI.meaning,I230.spare.val,I230.spare.const,I230.ModeSSSC.val,I230.ModeSSSC.meaning,I230.ARC.val,I230.ARC.meaning,I230.AIC.val,I230.AIC.meaning,I230.BDS16.val,I230.BDS37.val,ISP.SP.val,I020.TST.val,I020.TST.meaning,I020.spare.val,I020.spare.const,I020.XPP.val,I020.XPP.meaning,I020.ME.val,I020.ME.meaning,I020.MI.val,I020.MI.meaning,I020.FOEFRI.val,I020.FOEFRI.meaning,I170.TRE.val,I170.TRE.meaning,I170.GHO.val,I170.GHO.meaning,I170.SUP.val,I170.SUP.meaning,I170.TCC.val,I170.TCC.meaning,I170.spare.val,I170.spare.const,enu_e,enu_n,enu_u,wgs84_lat,wgs84_lon,wgs84_alt
0,48,1660671185101,132,17,1970-01-01 03:59:00,5,Single ModeS Roll-Call,0,Actual target report,0,Report from RDP Chain 1,0,Absence of SPI,0,Report from aircraft transponder,0,End of Data Item,28.773438,256.0,120.119019,0.0,Code validated,0.0,Default,0.0,Mode-3/A code as derived from the reply of the transponder,0.0,0.0,4465,0.0,Code validated,0.0,Default,15750.0,881222,NOK702,1.0,23008.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not active,0.0,Not active,0.0,Not active,0.0,0.0,No source information provided,0.0,Unknown,1.0,-1.054688,1.0,113.554688,1.0,248.0,0.0,0.00000,1.0,262.0,1.0,115.136719,1.0,197.0,1.0,0.400,1.0,1312.0,1.0,1312.0,3,1603.0,24.789062,-14.390625,249.609375,112.631836,0.0,Confirmed Track,2.0,SSR/ModeS Track,0.0,Normal confidence,0.0,No horizontal man. sensed,1.0,Climbing,0.0,End of Data Item,2.0,0.0,0.0,SI-Code Capable,0.0,0.0,1.0,Yes,1.0,25ft resolution,1.0,Yes,1.0,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45909.34375,-26651.43750,4800.60,18.518570,99.401494,5310.209492
1,48,1660671185102,132,17,1970-01-01 03:59:00,5,Single ModeS Roll-Call,0,Actual target report,1,Report from RDP Chain 2,0,Absence of SPI,0,Report from aircraft transponder,0,End of Data Item,28.773438,256.0,120.119019,0.0,Code validated,0.0,Default,0.0,Mode-3/A code as derived from the reply of the transponder,0.0,0.0,4465,0.0,Code validated,0.0,Default,15750.0,881222,NOK702,1.0,23008.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not active,0.0,Not active,0.0,Not active,0.0,0.0,No source information provided,0.0,Unknown,1.0,-1.054688,1.0,113.554688,1.0,248.0,0.0,0.00000,1.0,262.0,1.0,115.136719,1.0,197.0,1.0,0.400,1.0,1312.0,1.0,1

In [ ]:
#@title site coodinate in wgs84/height in meters
sitecoords_lat =18.759655#@param {type:"number"}
sitecoords_lon =98.967067#@param {type:"number"}
#site hight in meters
sitecoords_h=288.61981#@param {type:"number"}

In [61]:
#@title Filter SIC/SAC
SIC=17#@param {type:"integer"}
SAC=132#@param {type:"integer"}
df_site=df_reduced.loc[df_reduced['I010.SIC.val'] == SIC]
df_site=df_site.loc[df_site['I010.SAC.val'] == 132]
df_site=df_site[df_site['I042.X.val'].notnull()]
lat,lon,alt=pm.enu.enu2geodetic(df_site["enu_e"], df_site["enu_n"], df_site["enu_u"], sitecoords_lat, sitecoords_lon, sitecoords_h, ell=None, deg=True)
df_site["wgs84_lat"]=lat
df_site["wgs84_lon"]=lon
df_site["wgs84_alt"]=alt



In [60]:
#@title Filter missing ICAO
ICAO_filter = True #@param {type:"boolean"}
if ICAO_filter:
  I250_code=df_site[df_site['I220.ACAddr.val'].notnull()]
else:
  I250_code=df_site
I250_code.head()

,category,ts,I010.SAC.val,I010.SIC.val,I140.ToD.val,I020.TYP.val,I020.TYP.meaning,I020.SIM.val,I020.SIM.meaning,I020.RDP.val,I020.RDP.meaning,I020.SPI.val,I020.SPI.meaning,I020.RAB.val,I020.RAB.meaning,I020.FX.val,I020.FX.meaning,I040.RHO.val,I040.RHO.max,I040.THETA.val,I070.V.val,I070.V.meaning,I070.G.val,I070.G.meaning,I070.L.val,I070.L.meaning,I070.spare.val,I070.spare.const,I070.Mode3A.val,I090.V.val,I090.V.meaning,I090.G.val,I090.G.meaning,I090.FL.val,I220.ACAddr.val,I240.TId.val,I250.BDS_40.MCP_ALT_STATUS.val,I250.BDS_40.MCP_ALT.val,I250.BDS_40.FMS_ALT_STATUS.val,I250.BDS_40.FMS_ALT.val,I250.BDS_40.BP_STATUS.val,I250.BDS_40.BP.val,I250.BDS_40.res1.val,I250.BDS_40.MODE_STATUS.val,I250.BDS_40.VNAV.val,I250.BDS_40.VNAV.meaning,I250.BDS_40.ALT_HOLD.val,I250.BDS_40.ALT_HOLD.meaning,I250.BDS_40.APP.val,I250.BDS_40.APP.meaning,I250.BDS_40.res2.val,I250.BDS_40.TARGET_ALT_STATUS.val,I250.BDS_40.TARGET_ALT_STATUS.meaning,I250.BDS_40.TARGET_ALT_SOURCE.val,I250.BDS_40.TARGET_ALT_SOURCE.meaning,I250.BDS_50.RA_STATUS.val,I250.BDS_50.RA.val,I250.BDS_50.TTA_STATUS.val,I250.BDS_50.TTA.val,I250.BDS_50.GS_STATUS.val,I250.BDS_50.GS.val,I250.BDS_50.TAR_STATUS.val,I250.BDS_50.TAR.val,I250.BDS_50.TAS_STATUS.val,I250.BDS_50.TAS.val,I250.BDS_60.HDG_STATUS.val,I250.BDS_60.HDG.val,I250.BDS_60.IAS_STAT.val,I250.BDS_60.IAS.val,I250.BDS_60.MACH_STATUS.val,I250.BDS_60.MACH.val,I250.BDS_60.BAR_STATUS.val,I250.BDS_60.BAR.val,I250.BDS_60.IVV_STATUS.val,I250.BDS_60.IVV.val,I250.BDS_count,I161.Tn.val,I042.X.val,I042.Y.val,I200.CGS.val,I200.CHdg.val,I170.CNF.val,I170.CNF.meaning,I170.RAD.val,I170.RAD.meaning,I170.DOU.val,I170.DOU.meaning,I170.MAH.val,I170.MAH.meaning,I170.CDM.val,I170.CDM.meaning,I170.FX.val,I170.FX.meaning,I230.COM.val,I230.STAT.val,I230.SI.val,I230.SI.meaning,I230.spare.val,I230.spare.const,I230.ModeSSSC.val,I230.ModeSSSC.meaning,I230.ARC.val,I230.ARC.meaning,I230.AIC.val,I230.AIC.meaning,I230.BDS16.val,I230.BDS37.val,ISP.SP.val,I020.TST.val,I020.TST.meaning,I020.spare.val,I020.spare.const,I020.XPP.val,I020.XPP.meaning,I020.ME.val,I020.ME.meaning,I020.MI.val,I020.MI.meaning,I020.FOEFRI.val,I020.FOEFRI.meaning,I170.TRE.val,I170.TRE.meaning,I170.GHO.val,I170.GHO.meaning,I170.SUP.val,I170.SUP.meaning,I170.TCC.val,I170.TCC.meaning,I170.spare.val,I170.spare.const,enu_e,enu_n,enu_u,wgs84_lat,wgs84_lon,wgs84_alt
0,48,1660671185101,132,17,1970-01-01 03:59:00,5,Single ModeS Roll-Call,0,Actual target report,0,Report from RDP Chain 1,0,Absence of SPI,0,Report from aircraft transponder,0,End of Data Item,28.773438,256.0,120.119019,0.0,Code validated,0.0,Default,0.0,Mode-3/A code as derived from the reply of the transponder,0.0,0.0,4465,0.0,Code validated,0.0,Default,15750.0,881222,NOK702,1.0,23008.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not active,0.0,Not active,0.0,Not active,0.0,0.0,No source information provided,0.0,Unknown,1.0,-1.054688,1.0,113.554688,1.0,248.0,0.0,0.00000,1.0,262.0,1.0,115.136719,1.0,197.0,1.0,0.400,1.0,1312.0,1.0,1312.0,3,1603.0,24.789062,-14.390625,249.609375,112.631836,0.0,Confirmed Track,2.0,SSR/ModeS Track,0.0,Normal confidence,0.0,No horizontal man. sensed,1.0,Climbing,0.0,End of Data Item,2.0,0.0,0.0,SI-Code Capable,0.0,0.0,1.0,Yes,1.0,25ft resolution,1.0,Yes,1.0,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45909.34375,-26651.43750,4800.60,18.518570,99.401494,5310.209492
1,48,1660671185102,132,17,1970-01-01 03:59:00,5,Single ModeS Roll-Call,0,Actual target report,1,Report from RDP Chain 2,0,Absence of SPI,0,Report from aircraft transponder,0,End of Data Item,28.773438,256.0,120.119019,0.0,Code validated,0.0,Default,0.0,Mode-3/A code as derived from the reply of the transponder,0.0,0.0,4465,0.0,Code validated,0.0,Default,15750.0,881222,NOK702,1.0,23008.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Not active,0.0,Not active,0.0,Not active,0.0,0.0,No source information provided,0.0,Unknown,1.0,-1.054688,1.0,113.554688,1.0,248.0,0.0,0.00000,1.0,262.0,1.0,115.136719,1.0,197.0,1.0,0.400,1.0,1312.0,1.0,1

In [70]:
#@title Plot all flight
I250_code["I250.BDS_count"] = I250_code["I250.BDS_count"].astype(str)
fig = px.scatter(I250_code, x="I042.X.val", y="I042.Y.val", color="I250.BDS_count",
                 hover_data=['I240.TId.val','I090.FL.val',"I220.ACAddr.val","I040.RHO.val","I020.TYP.meaning"],
                 color_discrete_map={
                "0": "red",
                "1": "orange",
                "2": "yellow",
                "3": "green"})
fig.update_yaxes(range = [-210,210])
fig.update_xaxes(range = [-210,210])
fig.show()



In [73]:
fig = px.scatter_mapbox(I250_code, lat="wgs84_lat", lon="wgs84_lon", color="I250.BDS_count",hover_name="I220.ACAddr.val",
                 hover_data=['I140.ToD.val','I240.TId.val','I090.FL.val',"I040.RHO.val"],
                 color_discrete_map={
                "0": "red",
                "1": "orange",
                "2": "yellow",
                "3": "#0F0"}, zoom=3, height=300)
# fig.update_layout(mapbox_style="open-street-map")
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=4, mapbox_center_lat = sitecoords_lat,mapbox_center_lon = sitecoords_lon,
    margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#@title I250 altitude stats
I250_have=I250_code[I250_code['I250.BDS_count']!="0"]
I250_miss=I250_code[I250_code['I250.BDS_count']=="0"]
hflh=I250_have['I090.FL.val'].max()
hfll=I250_have['I090.FL.val'].min()
mflh=I250_miss['I090.FL.val'].max()
mfll=I250_miss['I090.FL.val'].min()
print("Have I250 @ altitude")
print("max:",hflh)
print("min:",hfll)
print("missing I250 @ altitude")
print("max:",mflh)
print("min:",mfll)

Have I250 @ altitude
max: 39025.0
min: 1000.0
missing I250 @ altitude
max: 39000.0
min: 1025.0


In [83]:
#@title Plot only selected ICAO
icao_list=list(set(I250_code["I220.ACAddr.val"]))
@interact
def plot_ICAO(icao=icao_list,style=["on_grid","on_map"]):
    df_sel_acaddr=I250_code.loc[I250_code['I220.ACAddr.val'] == icao]
    if style=="on_map":
        fig = px.scatter_mapbox(df_sel_acaddr, lat="wgs84_lat", lon="wgs84_lon", color="I250.BDS_count",hover_name="I220.ACAddr.val",
                        hover_data=['I140.ToD.val','I240.TId.val','I090.FL.val',"I040.RHO.val","I040.THETA.val"],
                        color_discrete_map={
                        "0": "red",
                        "1": "orange",
                        "2": "yellow",
                        "3": "#0F0"}, zoom=3, height=300)
        fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=4, mapbox_center_lat = sitecoords_lat,mapbox_center_lon = sitecoords_lon,
            margin={"r":0,"t":0,"l":0,"b":0})
        fig.show()
    else:
        fig = px.scatter(df_sel_acaddr, x="I042.X.val", y="I042.Y.val", color="I250.BDS_count",
                    hover_data=['I140.ToD.val','I240.TId.val','I090.FL.val',"I040.RHO.val","I040.THETA.val"],
                 color_discrete_map={
                "0": "red",
                "1": "orange",
                "2": "yellow",
                "3": "green"})
        fig.update_yaxes(range = [-210,210])
        fig.update_xaxes(range = [-210,210])
        fig.show()

interactive(children=(Dropdown(description='icao', options=('704016', '88104B', '885968', '881422', '884212', …